In [1]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))


################################################################################
## General Stuff
################################################################################
from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from urllib.parse import quote
from timeUtils import clock, elapsed
from fsUtils import moveFile, setFile, setDir, setSubDir, isFile, isDir, mkDir
from fileUtils import getFileBasics, getBasename
from webUtils import getHTML
from time import sleep
import time
from multiprocessing import Pool


######################################################
# Music Stuff
######################################################
from dbBase import dbBase
from multiArtist import multiartist
from myMusicDBMap import myMusicDBMap
from mainDB import mainDB
from parseDBArtistsData import parseDBArtistsData




import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

Python: 3.7.7 (default, Mar 26 2020, 10:32:53) 
[Clang 4.0.1 (tags/RELEASE_401/final)]


/Users/tgadfort/opt/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


Notebook Last Run Initiated: 2020-08-29 17:22:26.895068


In [3]:
%load_ext autoreload
%autoreload
from mainDB import mainDB
mdb    = myMusicDBMap(debug=True)
maindb = mainDB(mdb=mdb, create=True, debug=True)
pdb    = parseDBArtistsData(maindb=maindb)
dbs    = ['AllMusic', 'Discogs', 'MusicBrainz', 'LastFM']
#dbs    = ['AllMusic']
#dbs = ['MusicBrainz', 'LastFM']
#dbs = ["LastFM"]
for db in dbs:
    pdb.parse(db, nProcs=3, modVals=list(range(100)), force=False)
    #pdb.parse(db, nProcs=3, modVals=[0,1,2], force=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Creating myMusicDBMap()
   Loading my music db map: /Users/tgadfort/opt/anaconda3/envs/py37/musicdb/myMusicMap.p
   DB keys: ['Discogs', 'AllMusic', 'MusicBrainz', 'AceBootlegs', 'RateYourMusic', 'LastFM', 'DatPiff', 'RockCorner', 'CDandLP', 'MusicStack', 'MetalStorm']
Found 4206 artist entries
Counter({'Discogs': 4101, 'AllMusic': 3991, 'MusicBrainz': 3846, 'LastFM': 3228, 'CDandLP': 755, 'DatPiff': 545, 'RockCorner': 491, 'AceBootlegs': 178, 'RateYourMusic': 68, 'MusicStack': 5, 'MetalStorm': 0})
Counter({'Discogs': 4101, 'AllMusic': 3991, 'MusicBrainz': 3846, 'LastFM': 3228, 'CDandLP': 755, 'DatPiff': 545, 'RockCorner': 491, 'AceBootlegs': 178, 'RateYourMusic': 68, 'MusicStack': 5, 'MetalStorm': 0})
Setting Basic Database Objects
=========================   discogs   =========================
=========================   allmusic   =========================
=========================   musicbrainz 

  ===> Found 694 files. There are 85 new files to parse.
  ===> Found 694 files. There are 89 new files to parse.
  ===> Found 656 files. There are 60 new files to parse.
Saving 204 new artist IDs to /Volumes/Piggy/Discog/artists-allmusic-db/4-DB.p
  --> This file is 464.0kB.
Saving 544 new artist IDs name data to /Volumes/Piggy/Discog/artists-allmusic-db/metadata/4-Metadata.p
  --> This file is 25.4kB.
Saving 544 new artist IDs media data to /Volumes/Piggy/Discog/artists-allmusic-db/metadata/4-MediaMetadata.p
  --> This file is 275.2kB.
Parsing Artist Files For ModVal 5
Saving 215 new artist IDs to /Volumes/Piggy/Discog/artists-allmusic-db/13-DB.p
  --> This file is 421.2kB.
Saving 531 new artist IDs name data to /Volumes/Piggy/Discog/artists-allmusic-db/metadata/13-Metadata.p
  --> This file is 24.6kB.
Saving 531 new artist IDs media data to /Volumes/Piggy/Discog/artists-allmusic-db/metadata/13-MediaMetadata.p
  --> This file is 240.8kB.
Parsing Artist Files For ModVal 14
  ===> Foun

Saving 511 new artist IDs media data to /Volumes/Piggy/Discog/artists-allmusic-db/metadata/36-MediaMetadata.p
  --> This file is 270.8kB.
Parsing Artist Files For ModVal 37
  ===> Found 660 files. There are 91 new files to parse.
Saving 205 new artist IDs to /Volumes/Piggy/Discog/artists-allmusic-db/47-DB.p
  --> This file is 446.8kB.
Saving 540 new artist IDs name data to /Volumes/Piggy/Discog/artists-allmusic-db/metadata/47-Metadata.p
  --> This file is 25.0kB.
Saving 540 new artist IDs media data to /Volumes/Piggy/Discog/artists-allmusic-db/metadata/47-MediaMetadata.p
  --> This file is 258.8kB.
Parsing Artist Files For ModVal 48
  ===> Found 697 files. There are 76 new files to parse.
Saving 188 new artist IDs to /Volumes/Piggy/Discog/artists-allmusic-db/28-DB.p
  --> This file is 545.5kB.
Saving 545 new artist IDs name data to /Volumes/Piggy/Discog/artists-allmusic-db/metadata/28-Metadata.p
  --> This file is 25.1kB.
Saving 545 new artist IDs media data to /Volumes/Piggy/Discog/ar

Saving 219 new artist IDs to /Volumes/Piggy/Discog/artists-allmusic-db/33-DB.p
Saving 202 new artist IDs to /Volumes/Piggy/Discog/artists-allmusic-db/53-DB.p
  --> This file is 540.4kB.
Saving 562 new artist IDs name data to /Volumes/Piggy/Discog/artists-allmusic-db/metadata/33-Metadata.p
  --> This file is 26.1kB.
Saving 562 new artist IDs media data to /Volumes/Piggy/Discog/artists-allmusic-db/metadata/33-MediaMetadata.p
  --> This file is 455.4kB.
Saving 509 new artist IDs name data to /Volumes/Piggy/Discog/artists-allmusic-db/metadata/53-Metadata.p
  --> This file is 332.6kB.
  --> This file is 23.3kB.
Parsing Artist Files For ModVal 34
Saving 509 new artist IDs media data to /Volumes/Piggy/Discog/artists-allmusic-db/metadata/53-MediaMetadata.p
  --> This file is 269.7kB.
Parsing Artist Files For ModVal 54
  ===> Found 712 files, but there are no new files to parse so skipping.
Parsing Artist Files For ModVal 55
  ===> Found 719 files. There are 91 new files to parse.
  ===> Found 

  ===> Found 674 files. There are 70 new files to parse.
Saving 190 new artist IDs to /Volumes/Piggy/Discog/artists-allmusic-db/81-DB.p
Saving 192 new artist IDs to /Volumes/Piggy/Discog/artists-allmusic-db/74-DB.p
  --> This file is 510.8kB.
Saving 549 new artist IDs name data to /Volumes/Piggy/Discog/artists-allmusic-db/metadata/74-Metadata.p
  --> This file is 558.2kB.
  --> This file is 25.3kB.
Saving 549 new artist IDs media data to /Volumes/Piggy/Discog/artists-allmusic-db/metadata/74-MediaMetadata.p
Saving 548 new artist IDs name data to /Volumes/Piggy/Discog/artists-allmusic-db/metadata/81-Metadata.p
  --> This file is 25.4kB.
Saving 548 new artist IDs media data to /Volumes/Piggy/Discog/artists-allmusic-db/metadata/81-MediaMetadata.p
  --> This file is 349.1kB.  --> This file is 309.4kB.

Parsing Artist Files For ModVal 75Parsing Artist Files For ModVal 82

  ===> Found 668 files. There are 68 new files to parse.
  ===> Found 696 files. There are 77 new files to parse.
Saving 

...

Saving 72 new artist IDs to /Volumes/Piggy/Discog/artists-lastfm-db/13-DB.p
  --> This file is 318.6kB.
Saving 1181 new artist IDs name data to /Volumes/Piggy/Discog/artists-lastfm-db/metadata/13-Metadata.p
  --> This file is 79.7kB.
Saving 1181 new artist IDs media data to /Volumes/Piggy/Discog/artists-lastfm-db/metadata/13-MediaMetadata.p
  --> This file is 124.9kB.
Parsing Artist Files For ModVal 14
Saving 90 new artist IDs to /Volumes/Piggy/Discog/artists-lastfm-db/4-DB.p
  ===> Found 1119 files. There are 88 new files to parse.
  --> This file is 315.3kB.
Saving 1102 new artist IDs name data to /Volumes/Piggy/Discog/artists-lastfm-db/metadata/4-Metadata.p
  --> This file is 72.7kB.
Saving 1102 new artist IDs media data to /Volumes/Piggy/Discog/artists-lastfm-db/metadata/4-MediaMetadata.p
  --> This file is 131.5kB.
Parsing Artist Files For ModVal 5
  ===> Found 1127 files. There are 78 new files to parse.
Saving 68 new artist IDs to /Volumes/Piggy/Discog/artists-lastfm-db/23-DB.p


Saving 1102 new artist IDs media data to /Volumes/Piggy/Discog/artists-lastfm-db/metadata/28-MediaMetadata.p
  --> This file is 120.1kB.
Parsing Artist Files For ModVal 29
Saving 83 new artist IDs to /Volumes/Piggy/Discog/artists-lastfm-db/37-DB.p
  --> This file is 295.2kB.
Saving 1143 new artist IDs name data to /Volumes/Piggy/Discog/artists-lastfm-db/metadata/37-Metadata.p
  --> This file is 76.5kB.
Saving 1143 new artist IDs media data to /Volumes/Piggy/Discog/artists-lastfm-db/metadata/37-MediaMetadata.p
  --> This file is 111.7kB.
Parsing Artist Files For ModVal 38
  ===> Found 1074 files. There are 79 new files to parse.
  ===> Found 1110 files. There are 84 new files to parse.
  ===> Found 1144 files. There are 78 new files to parse.
Saving 79 new artist IDs to /Volumes/Piggy/Discog/artists-lastfm-db/29-DB.p
Saving 84 new artist IDs to /Volumes/Piggy/Discog/artists-lastfm-db/54-DB.p
  --> This file is 311.7kB.
Saving 1074 new artist IDs name data to /Volumes/Piggy/Discog/artist

  ===> Found 1064 files. There are 81 new files to parse.
  --> This file is 319.5kB.
Saving 1122 new artist IDs name data to /Volumes/Piggy/Discog/artists-lastfm-db/metadata/34-Metadata.p
  --> This file is 72.9kB.
Saving 1122 new artist IDs media data to /Volumes/Piggy/Discog/artists-lastfm-db/metadata/34-MediaMetadata.p
  --> This file is 131.9kB.
Parsing Artist Files For ModVal 35
  ===> Found 1129 files. There are 87 new files to parse.
Saving 91 new artist IDs to /Volumes/Piggy/Discog/artists-lastfm-db/43-DB.p
  --> This file is 306.5kB.
Saving 1084 new artist IDs name data to /Volumes/Piggy/Discog/artists-lastfm-db/metadata/43-Metadata.p
  --> This file is 72.3kB.
Saving 1084 new artist IDs media data to /Volumes/Piggy/Discog/artists-lastfm-db/metadata/43-MediaMetadata.p
  --> This file is 126.1kB.
Parsing Artist Files For ModVal 44
  ===> Found 1094 files. There are 86 new files to parse.
Saving 81 new artist IDs to /Volumes/Piggy/Discog/artists-lastfm-db/60-DB.p
  --> This fil

  --> This file is 116.4kB.
Parsing Artist Files For ModVal 77  --> This file is 121.3kB.

Parsing Artist Files For ModVal 93
  ===> Found 1140 files. There are 74 new files to parse.
  ===> Found 1140 files. There are 89 new files to parse.
Saving 74 new artist IDs to /Volumes/Piggy/Discog/artists-lastfm-db/93-DB.p
  --> This file is 314.3kB.
Saving 1140 new artist IDs name data to /Volumes/Piggy/Discog/artists-lastfm-db/metadata/93-Metadata.p
  --> This file is 75.2kB.
Saving 1140 new artist IDs media data to /Volumes/Piggy/Discog/artists-lastfm-db/metadata/93-MediaMetadata.p
  --> This file is 125.0kB.
Parsing Artist Files For ModVal 94
Saving 89 new artist IDs to /Volumes/Piggy/Discog/artists-lastfm-db/77-DB.p
  --> This file is 309.3kB.
Saving 1140 new artist IDs name data to /Volumes/Piggy/Discog/artists-lastfm-db/metadata/77-Metadata.p
  --> This file is 76.5kB.
Saving 1140 new artist IDs media data to /Volumes/Piggy/Discog/artists-lastfm-db/metadata/77-MediaMetadata.p
  --> Thi

ValueError: Cannot read artist file: /Volumes/Piggy/Discog/artists-lastfm/46/73898369646.p

In [8]:
pdb.parse(db="AllMusic", modVals=[15], nProcs=1, force=True)

Parsing AllMusic with 1 processes using [[15]] mod values.
Parsing Artist Files For ModVal 15
Forcing Reloads of ModVal=15
  Processing 611 files.
	Processed 0/611 files.
	Processed 500/611 files.
Saving 611 new artist IDs to /Volumes/Piggy/Discog/artists-allmusic-db/15-DB.p
  --> This file is 486.1kB.
Saving 477 new artist IDs name data to /Volumes/Piggy/Discog/artists-allmusic-db/metadata/15-Metadata.p
  --> This file is 21.9kB.
Saving 477 new artist IDs media data to /Volumes/Piggy/Discog/artists-allmusic-db/metadata/15-MediaMetadata.p
  --> This file is 301.7kB.



[None]

In [ ]:
nProcs = 8
%load_ext autoreload
%autoreload
from mainDB import mainDB
maindb = mainDB(mdb=None, create=True, debug=True)
start,cmt=clock("Parsing All Artists")
start2,cmt2=clock("Parallel Artist Parsing")

parseArtistsParallel(db="AllMusic", nProcs=nProcs)
maindb.setDBFull("AllMusic")

parseArtistsParallel(db="MusicBrainz", nProcs=nProcs)
maindb.setDBFull("MusicBrainz")

parseArtistsParallel(db="Discogs", nProcs=nProcs)
maindb.setDBFull("Discogs")

#parseArtistsParallel(db="AceBootlegs", nProcs=1)
#parseArtistsParallel(db="DatPiff", nProcs=1)
#parseArtistsParallel(db="RateYourMusic", nProcs=3)
parseArtistsParallel(db="LastFM", nProcs=nProcs)
maindb.setDBFull("LastFM")

#parseArtistsParallel(db="RockCorner", nProcs=3)
#parseArtistsParallel(db="CDandLP", nProcs=3)
#parseArtistsParallel(db="MusicStack", nProcs=1)
elapsed(start2, cmt2)
elapsed(start, cmt)

# Master DB

In [ ]:

print("\n\n{0}\n".format("Full DB"))
print("\n\n{0}\n".format("Known DB"))
#maindb.setDBKnown() ## Do this to recreate everything
#maindb.setDBKnown()
#artistDBs = maindb.getKnownArtistDBs()

# Download My Known Artists

In [ ]:
mdb = myMusicDBMap(debug=True)

In [ ]:
dbsToGet = ["AllMusic", "Discogs", "MusicBrainz", "LastFM", "RockCorner", "CDandLP"]
searches = [dbdata["AllMusic"]["Artists"].searchAllMusicForArtist, dbdata["Discogs"]["Artists"].searchDiscogForArtist,
            dbdata["MusicBrainz"]["Artists"].searchMusicBrainzForArtist, dbdata["LastFM"]["Artists"].searchLastFMForArtist,
            dbdata["RockCorner"]["Artists"].searchRockCornerForArtist, dbdata["CDandLP"]["Artists"].searchCDandLPForArtist]

dbsToGet = ["AllMusic"]
searches = [dbdata["AllMusic"]["Artists"].searchAllMusicForArtist]

In [ ]:
def searchAll(mdb, minI=-1):
    nArtists = len(mdb.getArtists())
    for i,artistName in enumerate(mdb.getArtists()):
        if i <= minI:
            continue

        print('\n\n\n')
        print('-'*130)
        print(i,'/',nArtists,'===>\t',artistName)
        print('-'*130)
        print('\n')
        
        dbdata = mdb.getArtistData(artistName)
        if all([dbdata.get("LastFM"), dbdata.get("Discogs"), dbdata.get("AllMusic"), dbdata.get("MusicBrainz")]):
            print("{0} is well known.".format(artistName))
            continue
        else:
            print("{0} get it!".format(artistName))

        for search in searches:
            print("====>",search)
            try:
                search(artistName)
            except:
                sleep(3)

In [ ]:
searchAll(mdb, 3404)

In [ ]:
# Refused

# Albums

In [ ]:
%load_ext autoreload
%autoreload
from album import album
from albums import albums
from discogsBase import discogs

disc = discogs()
albs = albums(disc)
#albs.parseAlbumModValFiles(modVal=27)

In [ ]:
def parseAlbums(modVal):
    albs.parseAlbumModValFiles(modVal=modVal)
    #albs.createAlbumModValMetadata(modVal)
    #albs.createAlbumModValMetadata(modVal=modVal)

def parseAlbumsParallel(modVals=None):
    if modVals is None:
        modVals = ["NAN"] + list(range(100))
        #modVals = range(100)
    nProcs = 2
    pool = Pool(processes=nProcs)
    result = pool.map_async(parseAlbums, modVals)
 
    start, cmt = clock("Processing {0} modVals using {1} processes".format(modVals, nProcs))
    print("Runing .",end="")
    while not result.ready():
        #print(".",end="")
        time.sleep(10)
    print("")
    
    elapsed(start, cmt)
    return result.get()

In [ ]:
%load_ext autoreload
%autoreload
from album import album
from albums import albums
albs = albums(disc)

start,cmt=clock("Parsing All Albums")
start2,cmt2=clock("Parallel Album Parsing")
parseAlbumsParallel()
elapsed(start2, cmt2)
elapsed(start, cmt)

# Download Stuff

In [ ]:
mdb = myMusicDBMap(debug=True)
maindb = mainDB(mdb=mdb, create=False, debug=True)
dbdata = maindb.dbdata

In [ ]:
ifile = "unmatchedArtists4.p"
noMatches = getFile(ifile)
print(len(noMatches))
dbs = ['Discogs', 'AllMusic', 'MusicBrainz', 'LastFM']
dbs = ['MusicBrainz']

#noMatches=['Go-A', 'PM', 'Qango']

from time import sleep
maxI = -1
for i,artist in enumerate(noMatches):
    print("="*100)
    print("="*100)
    print("="*100)
    print("="*100)
    print(i,'/',len(noMatches),'\t',artist)
    if i <= maxI:
        continue
    for db in dbs:
        print("="*100)
        print(i,'/',len(noMatches),'\t',artist,'===>',db)
        try:
            dbdata[db]["Artists"].searchForArtist(artist)
        except:
            sleep(5)